In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from pathlib import Path, PurePath
import json
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
import re

In [2]:
# path of note sound files
pth1 = Path('.') / 'data'
pth2 = Path('.') / '..' / 'note-spider' / 'data'
data_pth = pth2 if pth2.exists() else pth1
if not data_pth.exists():
    raise('Cannot find data')
data_pth

PosixPath('../note-spider/data')

In [3]:
file_pth_json = None
with open(data_pth / 'urlsAndPath.json') as f:
    text = f.read()
    file_pth_json = json.loads(text)
file_pth_json[:3]

[{'url': 'https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.Bb3B3.aiff',
  'path': '/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default'},
 {'url': 'https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C4B4.aiff',
  'path': '/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default'},
 {'url': 'https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C5B5.aiff',
  'path': '/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default'}]

In [4]:
file_pth_dict = { 'url': [o['url'] for o in file_pth_json], 'path': [o['path'] for o in file_pth_json] }
df = pd.DataFrame(file_pth_dict, columns=['url','path', 'local_path'], index=None)
df.head()

,url,path,local_path
0,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.Bb3B3.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,NaN
1,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C4B4.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,NaN
2,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C5B5.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,NaN
3,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C6Ab6.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,NaN
4,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.mf.Bb3B3.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,NaN


In [5]:
def extend_col_local_path(row):
    url, path = row['url'], row['path']
    file_name = url.split('/')[-1]
    file_path = path + '/' + file_name
    relative_path =  file_path.replace('/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data','.')
    return str(data_pth.joinpath(relative_path))

def extend_col_relative_path(row):
    return Path(row['local_path']).relative_to(data_pth)

In [6]:
df['local_path'] = df.apply(extend_col_local_path, axis=1)
df['relative_path'] = df.apply(extend_col_relative_path, axis=1)

In [7]:
df.head()

,url,path,local_path,relative_path
0,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.Bb3B3.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,../note-spider/data/pre2012/woodwinds/oboe/default/Oboe.pp.Bb3B3.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.Bb3B3.aiff
1,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C4B4.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,../note-spider/data/pre2012/woodwinds/oboe/default/Oboe.pp.C4B4.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.C4B4.aiff
2,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C5B5.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,../note-spider/data/pre2012/woodwinds/oboe/default/Oboe.pp.C5B5.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.C5B5.aiff
3,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C6Ab6.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,../note-spider/data/pre2012/woodwinds/oboe/default/Oboe.pp.C6Ab6.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.C6Ab6.aiff
4,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.mf.Bb3B3.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,../note-spider/data/pre2012/woodwinds/oboe/default/Oboe.mf.Bb3B3.aiff,pre2012/woodwinds/oboe/default/Oboe.mf.Bb3B3.aiff


In [8]:
def check_local_path_exists(row):
    local_path = Path(row['local_path'])
    if not local_path.exists():
         print(row)
    return local_path.exists()
df.apply(check_local_path_exists, axis=1)

url                                                                   https://theremin.music.uiowa.edu/oops.html
path             /Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/piano/piano/default
local_path                                             ../note-spider/data/pre2012/piano/piano/default/oops.html
relative_path                                                              pre2012/piano/piano/default/oops.html
Name: 753, dtype: object


0       True
1       True
2       True
3       True
4       True
        ... 
3682    True
3683    True
3684    True
3685    True
3686    True
Length: 3687, dtype: bool

### Extract path info
group: woodwinds
name: oboe
type: 
dynamics: pp, mf , ff

### 1
splash.bow.aif
20chinese.bow.aif

> No explicit meaning, skip processing

### 2
Oboe.pp.Bb3B3.aiff

> dynamics.note

20windgong.bow.stereo.aif

### 3

*Case 1*: Flute.vib.pp.B3B4.aiff

*Case 2*: Vibraphone.bow.Db6.stereo.aif


### 4
Viola.arco.sulC.pp.C3B3.aiff

Vibraphone.shortsustain.ff.F6.stereo.aif


### 5

Bass.arco.ff.sulE.E1.stereo.aif

Xylophone.rosewood.roll.ff.Ab7.stereo.aif

In [9]:
def get_path_info(row):
    relative_path = row['relative_path']
    splited = str(relative_path).split('/')
    year, group, name, subtype = splited[:-1]
    return splited[:-1]

def extend_col_year(row):
    return get_path_info(row)[0]

def extend_col_group(row):
    return get_path_info(row)[1]

def extend_col_name(row):
    return get_path_info(row)[2]

def extend_col_subtype(row):
    subtype = get_path_info(row)[3]
    return subtype if subtype != 'default' else None

In [10]:
df['year'] = df.apply(extend_col_year, axis=1)
df['group'] = df.apply(extend_col_group, axis=1)
df['instrument'] = df.apply(extend_col_name, axis=1)
df['subtype'] = df.apply(extend_col_subtype, axis=1)

#### Extract dynamics, notes

In [11]:
def find_dynamics(strs):
    result = []
    for s in strs:
        res = re.findall(r"^[pmf]{1,3}$",s)
        result.extend(res)
    return result

def find_note(strs):
    result = []
    for s in strs:
        res = re.findall(r"^[A-G]{1,}[A-Gb0-9]{1,}$",s)
        result.extend(res)
    return result
def extend_col_dynamics(row):
    file_path = Path(row['local_path'])
    file_name = file_path.name
    splited = file_name.split('.')
    dynamics = find_dynamics(splited)
    if len(dynamics) > 1:
        print('Wrong res', dynamics)
    if len(dynamics) == 0:
        return None
    return dynamics[0]

def extend_col_note(row):
    file_path = Path(row['local_path'])
    file_name = file_path.name
    splited = file_name.split('.')
    note = find_note(splited)
    if len(note) > 1:
        print('Wrong res', note)
    if len(note) == 0:
        return None
    return note[0]

def extend_col_notes(row):
    note = row['note']
    if note is None:
        return note
    return re.findall(r"[A-G]b?[0-9]", note)

In [12]:
df['dynamics'] = df.apply(extend_col_dynamics, axis=1)
df['note'] = df.apply(extend_col_note, axis=1)
df['notes'] = df.apply(extend_col_notes, axis=1)

In [13]:
df[['dynamics', 'note','notes', 'group', 'instrument','subtype']].head()

,dynamics,note,notes,group,instrument,subtype
0,pp,Bb3B3,"[Bb3, B3]",woodwinds,oboe,None
1,pp,C4B4,"[C4, B4]",woodwinds,oboe,None
2,pp,C5B5,"[C5, B5]",woodwinds,oboe,None
3,pp,C6Ab6,"[C6, Ab6]",woodwinds,oboe,None
4,mf,Bb3B3,"[Bb3, B3]",woodwinds,oboe,None


### Tags

In [14]:
def extend_col_tags(row):
    group,name, subtype = row['group'],row['instrument'],row['subtype']
    note, notes = row['note'], row['notes']
    alltags = [group, name]
    if subtype is not None:
        alltags.append(subtype)
    if note is not None:
        alltags.append(note)
        alltags.extend(notes)
    return [*set(alltags)]
    

In [15]:
df['tags'] = df.apply(extend_col_tags, axis=1)

In [16]:
df.head()

,url,path,local_path,relative_path,year,group,instrument,subtype,dynamics,note,notes,tags
0,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.Bb3B3.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,../note-spider/data/pre2012/woodwinds/oboe/default/Oboe.pp.Bb3B3.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.Bb3B3.aiff,pre2012,woodwinds,oboe,None,pp,Bb3B3,"[Bb3, B3]","[Bb3, Bb3B3, B3, oboe, woodwinds]"
1,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C4B4.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,../note-spider/data/pre2012/woodwinds/oboe/default/Oboe.pp.C4B4.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.C4B4.aiff,pre2012,woodwinds,oboe,None,pp,C4B4,"[C4, B4]","[C4B4, C4, B4, oboe, woodwinds]"
2,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C5B5.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,../note-spider/data/pre2012/woodwinds/oboe/default/Oboe.pp.C5B5.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.C5B5.aiff,pre2012,woodwinds,oboe,None,pp,C5B5,"[C5, B5]","[C5B5, B5, oboe, woodwinds, C5]"
3,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C6Ab6.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,../note-spider/data/pre2012/woodwinds/oboe/default/Oboe.pp.C6Ab6.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.C6Ab6.aiff,pre2012,woodwinds,oboe,None,pp,C6Ab6,"[C6, Ab6]","[Ab6, C6Ab6, C6, oboe, woodwinds]"
4,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.mf.Bb3B3.aiff,/Users/origami/Desktop/temp_projects/muse/packages/note-spider/data/pre2012/woodwinds/oboe/default,../note-spider/data/pre2012/woodwinds/oboe/default/Oboe.mf.Bb3B3.aiff,pre2012/woodwinds/oboe/default/Oboe.mf.Bb3B3.aiff,pre2012,woodwinds,oboe,None,mf,Bb3B3,"[Bb3, B3]","[Bb3, Bb3B3, B3, oboe, woodwinds]"


In [17]:
df2 = df.drop(['path','local_path'],axis=1)

In [18]:
df2.head()

,url,relative_path,year,group,instrument,subtype,dynamics,note,notes,tags
0,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.Bb3B3.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.Bb3B3.aiff,pre2012,woodwinds,oboe,None,pp,Bb3B3,"[Bb3, B3]","[Bb3, Bb3B3, B3, oboe, woodwinds]"
1,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C4B4.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.C4B4.aiff,pre2012,woodwinds,oboe,None,pp,C4B4,"[C4, B4]","[C4B4, C4, B4, oboe, woodwinds]"
2,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C5B5.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.C5B5.aiff,pre2012,woodwinds,oboe,None,pp,C5B5,"[C5, B5]","[C5B5, B5, oboe, woodwinds, C5]"
3,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C6Ab6.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.C6Ab6.aiff,pre2012,woodwinds,oboe,None,pp,C6Ab6,"[C6, Ab6]","[Ab6, C6Ab6, C6, oboe, woodwinds]"
4,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.mf.Bb3B3.aiff,pre2012/woodwinds/oboe/default/Oboe.mf.Bb3B3.aiff,pre2012,woodwinds,oboe,None,mf,Bb3B3,"[Bb3, B3]","[Bb3, Bb3B3, B3, oboe, woodwinds]"


In [19]:
df2 = df2.astype({
    'url':'string',
    'relative_path':'string',
    'year':'string',
    'group':'string',
    'instrument':'string',
    'subtype':'string',
    'dynamics':'string',
    'note':'string'
})
df2.dtypes

url              string
relative_path    string
year             string
group            string
instrument       string
subtype          string
dynamics         string
note             string
notes            object
tags             object
dtype: object

In [20]:
df2.head()

,url,relative_path,year,group,instrument,subtype,dynamics,note,notes,tags
0,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.Bb3B3.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.Bb3B3.aiff,pre2012,woodwinds,oboe,<NA>,pp,Bb3B3,"[Bb3, B3]","[Bb3, Bb3B3, B3, oboe, woodwinds]"
1,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C4B4.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.C4B4.aiff,pre2012,woodwinds,oboe,<NA>,pp,C4B4,"[C4, B4]","[C4B4, C4, B4, oboe, woodwinds]"
2,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C5B5.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.C5B5.aiff,pre2012,woodwinds,oboe,<NA>,pp,C5B5,"[C5, B5]","[C5B5, B5, oboe, woodwinds, C5]"
3,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.pp.C6Ab6.aiff,pre2012/woodwinds/oboe/default/Oboe.pp.C6Ab6.aiff,pre2012,woodwinds,oboe,<NA>,pp,C6Ab6,"[C6, Ab6]","[Ab6, C6Ab6, C6, oboe, woodwinds]"
4,https://theremin.music.uiowa.edu/sound files/MIS/Woodwinds/oboe/Oboe.mf.Bb3B3.aiff,pre2012/woodwinds/oboe/default/Oboe.mf.Bb3B3.aiff,pre2012,woodwinds,oboe,<NA>,mf,Bb3B3,"[Bb3, B3]","[Bb3, Bb3B3, B3, oboe, woodwinds]"


In [24]:
df2 = df2[~df2['relative_path'].str.contains('html')]

In [25]:
df2 = df2.reset_index()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3686 entries, 0 to 3685
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          3686 non-null   int64 
 1   url            3686 non-null   string
 2   relative_path  3686 non-null   string
 3   year           3686 non-null   string
 4   group          3686 non-null   string
 5   instrument     3686 non-null   string
 6   subtype        2935 non-null   string
 7   dynamics       3545 non-null   string
 8   note           3258 non-null   string
 9   notes          3258 non-null   object
 10  tags           3686 non-null   object
dtypes: int64(1), object(2), string(8)
memory usage: 316.9+ KB


In [26]:
df2.to_csv('output.csv')